In [1]:

from pyspark.sql import functions as F

In [2]:
from src.utils.spark_session import make_spark

spark = make_spark()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/30 09:30:37 WARN Utils: Your hostname, DSLSERVER00, resolves to a loopback address: 127.0.1.1; using 172.20.72.17 instead (on interface eno1)
25/09/30 09:30:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 09:30:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
paths = {
    "LISTINGS_RAW_PATH": "/home/hygo2025/Documents/data/listings",
    "EVENTS_RAW_PATH": "/home/hygo2025/Documents/data/events",
    "EVENTS_PROCESSED_PATH": "/home/hygo2025/Documents/data/processed_data/events",
    "LISTINGS_PROCESSED_PATH": "/home/hygo2025/Documents/data/processed_data/listings",
    "ENRICHED_EVENTS_PATH": "/home/hygo2025/Documents/data/processed_data/enriched_events",
    "USER_SESSIONS_PATH": "/home/hygo2025/Documents/data/processed_data/user_sessions",
    "LISTING_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/listing_id_mapping",
    "USER_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/user_id_mapping",
    "ANONYMOUS_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/anonymous_id_mapping",
    "JAVA_HOME": "/opt/jdk/amazon-corretto-21",
    "PYTHONUNBUFFERED": "1",
}


In [9]:
listings_raw = spark.read.option("mergeSchema", "true").parquet(paths["LISTINGS_PROCESSED_PATH"])

In [4]:
raw_events_raw = spark.read.option("mergeSchema", "true").parquet(paths["EVENTS_PROCESSED_PATH"])

In [5]:
raw_events_raw.columns


['anonymized_user_id',
 'anonymized_listing_id',
 'anonymized_anonymous_id',
 'anonymized_session_id',
 'dt',
 'browser_family',
 'os_family',
 'is_bot',
 'name_raw',
 'event_type',
 'collector_timestamp',
 'event_ts',
 'listing_id_numeric',
 'user_id_numeric']

In [7]:
raw_events_raw.filter(F.col("anonymized_user_id").isNull()).count()

0

In [11]:
listings_raw.count()

306146436

In [20]:
a = listings_raw.filter(F.col("anonymized_listing_id") == "259B932FD4548C22DD4CFCEA46D760253A97E967A637F34A81402AAE58C8B387")
print(a.show(10, truncate=False))

+----------------------------------------------------------------+----------+----------+------+---------+--------------+-------+--------------+------------+-----------+---------+--------+------+--------------+------+--------------+---------------------------+-------+--------------+------------------+----------+
|anonymized_listing_id                                           |created_at|updated_at|status|price    |state         |city   |neighborhood  |usable_areas|total_areas|bathrooms|bedrooms|suites|parking_spaces|floors|ceiling_height|amenities                  |month  |partition_date|listing_id_numeric|dt        |
+----------------------------------------------------------------+----------+----------+------+---------+--------------+-------+--------------+------------+-----------+---------+--------+------+--------------+------+--------------+---------------------------+-------+--------------+------------------+----------+
|259B932FD4548C22DD4CFCEA46D760253A97E967A637F34A81402AAE58C8

In [14]:
listings_raw.columns

['anonymized_listing_id',
 'created_at',
 'updated_at',
 'status',
 'price',
 'state',
 'city',
 'neighborhood',
 'usable_areas',
 'total_areas',
 'bathrooms',
 'bedrooms',
 'suites',
 'parking_spaces',
 'floors',
 'ceiling_height',
 'amenities',
 'month',
 'partition_date',
 'listing_id_numeric',
 'dt']